In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 29.2 MB/s 
     |████████████████████████████████| 212 kB 17.4 MB/s 
     |████████████████████████████████| 163 kB 12.1 MB/s 
     |████████████████████████████████| 115 kB 24.3 MB/s 
     |████████████████████████████████| 127 kB 22.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import TextVectorization
import tensorflow as tf
import os
from keras.layers import Embedding
from keras.initializers import Constant
from keras import layers, Input, Model

In [ ]:
dataset = load_dataset("SalehAhmad/Spam-Ham")
X = dataset['train']['sms_text']
Y = dataset['train']['label']

texts = []
labels = []
for i, label in enumerate(dataset['train']['label']):
    texts.append(dataset['train']['sms_text'][i])
    if label == 'ham':
        labels.append(0)
    else:
        labels.append(1)


print("number of texts :" , len(texts))
print("number of labels: ", len(labels))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/SalehAhmad___csv/SalehAhmad--Spam-Ham-3b83475258cf549a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

number of texts : 5566
number of labels:  5566


In [ ]:
texts_train, texts_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=1)

In [ ]:
#Create our Text Vectorizer to index our vocabulary based on the train samples 

vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=100)
text_ds = tf.data.Dataset.from_tensor_slices(texts_train).batch(128) ## Read batches of 128 samples
vectorizer.adapt(text_ds)

In [ ]:
#Print out top five words in the vocab
print(len(vectorizer.get_vocabulary())) ## We set max_tokens=10000
vectorizer.get_vocabulary()[:5]
output = vectorizer([["I feel good today"]])
output.numpy()[0, :4]

7802


array([  3, 191,  59,  87])

In [ ]:
## Create a map to get the unique list of the vocabulary
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

## print the unique list of integers for the same string using the new map "Word_index"
test = ["i", "feel", "good", "today"]
[word_index[w] for w in test]

[3, 191, 59, 87]

In [ ]:
type(y_train)

list

In [ ]:
## Vectorize our data (Convert the string data to integer data)
x_train = vectorizer(np.array([[s] for s in texts_train])).numpy()
y_train = np.array(y_train)

In [ ]:
y_train.shape

(3896,)

In [ ]:
## Download and unzip the Stanford GloVe model (pretrained word embeddings)
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-10-09 16:32:00--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-10-09 16:32:00--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-10-09 16:32:00--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
##10. Read the embeddings in the pretrained model (we are using the 100D version of GloVe)

path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
##11. Create "embedding_matrix" to index our vocabulary using the GloVe model 
num_tokens = len(voc) 
embedding_dim = 100 ## 100 dimensions
hits = 0 ## number of words that were found in the pretrained model
misses = 0 ## number of words that were missing in the pretrained model

# Prepare embedding matrix for our word list
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 5349 words (2453 misses)


In [ ]:
##12. Define our embedding layer for the training model 
## We load our embedding_matrix as the initializer and set trainable to False to avoid retraining this layer

embedding_layer = Embedding(num_tokens, embedding_dim,
                            embeddings_initializer= Constant(embedding_matrix), 
                            trainable=False,
)

In [ ]:
##13. Create a simple Bidirectional LSTM model

int_sequences_input = Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Bidirectional(layers.LSTM(20, return_sequences=True))(embedded_sequences)
x = layers.Bidirectional(layers.LSTM(20))(x)
preds = layers.Dense(2, activation="softmax")(x)
model = Model(int_sequences_input, preds)
model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_6 (Embedding)     (None, None, 100)         780200    
                                                                 
 bidirectional_16 (Bidirecti  (None, None, 40)         19360     
 onal)                                                           
                                                                 
 bidirectional_17 (Bidirecti  (None, 40)               9760      
 onal)                                                           
                                                                 
 dense_8 (Dense)             (None, 2)                 82        
                                                                 
Total params: 809,402
Trainable params: 29,202
Non-trainab

In [ ]:
##14. Train the model 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit(x_train, y_train, batch_size=128, epochs=20)

Epoch 1/20
31/31 [==============================] - 9s 119ms/step - loss: 0.4536 - acc: 0.8301
Epoch 2/20
31/31 [==============================] - 4s 122ms/step - loss: 0.2625 - acc: 0.8927
Epoch 3/20
31/31 [==============================] - 4s 121ms/step - loss: 0.1453 - acc: 0.9507
Epoch 4/20
31/31 [==============================] - 4s 121ms/step - loss: 0.1091 - acc: 0.9666
Epoch 5/20
31/31 [==============================] - 4s 119ms/step - loss: 0.0929 - acc: 0.9710
Epoch 6/20
31/31 [==============================] - 4s 121ms/step - loss: 0.0810 - acc: 0.9772
Epoch 7/20
31/31 [==============================] - 4s 121ms/step - loss: 0.0702 - acc: 0.9810
Epoch 8/20
31/31 [==============================] - 4s 119ms/step - loss: 0.0918 - acc: 0.9674
Epoch 9/20
31/31 [==============================] - 4s 121ms/step - loss: 0.0696 - acc: 0.9787
Epoch 10/20
31/31 [==============================] - 4s 120ms/step - loss: 0.0562 - acc: 0.9843
Epoch 11/20
31/31 [==============================

In [ ]:
##15. Test the model on a sample tweet from the test split

## Export a Model object to read a string of arbitary length
string_input = Input(shape=(1,), dtype="string")
x = vectorizer(string_input) 
preds = model(x)
end_to_end_model = Model(string_input, preds)

## Predict using the model
probabilities = end_to_end_model.predict([texts_test[10]])

print("String: {}".format(texts_test[10]))
print("Target output: {}".format(y_test[10]))
print("Predicted output: {}".format(np.argmax(probabilities[0])))

String: Ok... The theory test? when are ü going to book? I think it's on 21 may. Coz thought wanna go out with jiayin. But she isnt free
Target output: 0
Predicted output: 0


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

predLSTM = end_to_end_model.predict(texts_test)
pred_label = []
for i in predLSTM:
    pred_label.append(np.argmax(i))
print(confusion_matrix(pred_label, y_test))
print('Precision for LSTM: %.4f'% precision_score(y_test, pred_label,average='macro'))
print('Recall for LSTM: %.4f' % recall_score(y_test, pred_label, average='macro'))

[[1445   19]
 [   8  198]]
Precision for LSTM: 0.9741
Recall for LSTM: 0.9535


In [ ]:
# Word2vec
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gensim
w2v_model = gensim.models.Word2Vec(vector_size=embedding_dim, window=3, min_count=5, workers=8)

In [ ]:
documents=[text.split() for text in texts_train]
print(len(documents))
w2v_model.build_vocab(documents)

3896


In [ ]:
from gensim.models import KeyedVectors
embedding_matrix2 = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
  if word in w2v_model.wv:
    embedding_matrix2[i] = w2v_model.wv[word]

In [ ]:
##12. Define our embedding layer for the training model 
## We load our embedding_matrix as the initializer and set trainable to False to avoid retraining this layer
embedding_layer = Embedding(num_tokens, embedding_dim, weights=[embedding_matrix2], input_length=100, trainable=False)

In [ ]:
int_sequences_input2 = Input(shape=(None,), dtype="int64")
embedded_sequences2 = embedding_layer(int_sequences_input2)
x2 = layers.Bidirectional(layers.LSTM(20, return_sequences=True))(embedded_sequences2)
x2 = layers.Bidirectional(layers.LSTM(20))(x2)
preds2 = layers.Dense(2, activation="softmax")(x2)
model2 = Model(int_sequences_input2, preds2)
model2.summary()
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model2.fit(x_train, y_train, batch_size=128, epochs=20)
string_input2 = Input(shape=(1,), dtype="string")
x = vectorizer(string_input2) 
preds2 = model2(x)
end_to_end_model = Model(string_input2, preds2)
predLSTM = end_to_end_model.predict(texts_test)
pred_label = []
for i in predLSTM:
    pred_label.append(np.argmax(i))
print(confusion_matrix(pred_label, y_test))
print('Precision for LSTM: %.4f'% precision_score(y_test, pred_label,average='macro'))
print('Recall for LSTM: %.4f' % recall_score(y_test, pred_label, average='macro'))

Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_17 (Embedding)    (None, None, 100)         780200    
                                                                 
 bidirectional_42 (Bidirecti  (None, None, 40)         19360     
 onal)                                                           
                                                                 
 bidirectional_43 (Bidirecti  (None, 40)               9760      
 onal)                                                           
                                                                 
 dense_21 (Dense)            (None, 2)                 82        
                                                                 
Total params: 809,402
Trainable params: 29,202
Non-trainab